<a href="https://colab.research.google.com/github/dtabuena/CrispyMice/blob/main/Run_All_Crispy_Recs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
'Get Standard Modules'

import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import scipy
from scipy import stats
import os
from scipy.signal import butter,filtfilt
from tqdm import tqdm
import warnings
from IPython.display import clear_output
from datetime import datetime
import sys
import warnings
import shutil
from google.colab import files
warnings.filterwarnings('ignore')
np.set_printoptions(threshold=sys.maxsize)
clear_output(wait=False)



!pip install openpyxl
!pip install XlsxWriter


'''Get Repositories'''
try: shutil.rmtree('/content/EphysLib')
except: None

"run dtabuena's ephys notebooks"
!git clone https://github.com/dtabuena/EphysLib
to_import = [
            'ABF_Quality_Control.ipynb',
            'Basic_Ephys.ipynb',
            'Simple_ABF_tools.ipynb',
            'fun_math.ipynb',
            'importing_abfs_from_dropbox.ipynb',
            'QC_recoding_dataframe.ipynb',
            'Analyzers/input_resistance_analyzer.ipynb',
            'Analyzers/gain_analyzer.ipynb',
            'Analyzers/latencey_analyzer.ipynb',
            'Analyzers/IV_analyzer.ipynb',
            'Analyzers/Vm_analyzer.ipynb',
            'Analyzers/membrane_analyzer.ipynb',
            'Analyzers/rheobase_analyzer.ipynb',
            ]
for i in to_import:
    f = '/content/EphysLib/' + i
    %run $f


try: shutil.rmtree('/content/CrispyMice')
except: None
!git clone https://github.com/dtabuena/CrispyMice
to_import = ['CrispyPipelineFuncs.ipynb']
for i in to_import:
    f = '/content/CrispyMice/' + i
    %run $f


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 149 kB 9.7 MB/s 
Cloning into 'EphysLib'...
remote: Enumerating objects: 535, done.
remote: Counting objects: 100% (228/228), done.
remote: Compressing objects: 100% (103/103), done.
remote: Total 535 (delta 164), reused 171 (delta 125), pack-reused 307
Receiving objects: 100% (535/535), 4.57 MiB | 31.22 MiB/s, done.
Resolving deltas: 100% (338/338), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 53 kB 1.5 MB/s 
Cloning into 'CrispyMice'...
remote: Enumerating objects: 227, done.
remote: Counting objects: 100% (120/120), done.
remote: Compressing objects: 100% (95/95), done.
remote: Total 227 (delta 85), reused 25 (delta 25), pack-reused 107
Receiving obje

In [2]:
##### Setup Protocol List
VC_prot = ['VC - MemTest-10ms-160ms',
           'VC - Multi IV - 150ms',]
IC_prot = ['IC - Gain - D20pA',
           'IC - Gain - D50pA',
           'IC - Rheobase',
           'IC - R input',
           'IC - Latentcy 800pA-1s'
           'VC - 3min GapFree']

In [ ]:
dataset = {'data_name': 'RNF182',
           'data_source': "https://www.dropbox.com/sh/n9t8p257wnzlijk/AAC9Z36JodisyZjnM3mkJC3Xa?dl=0",
           'file_naming_scheme': ['Rec_date','Virus','GenoType','Sex','Age','Slice_Num','Cell_num','Cell_Type'],
           }

RNF182_results = crispy_mouse_wrap(dataset,VC_prot,IC_prot,strat_cols=['Rec_date', 'Cell_Type'])

In [ ]:
dataset = {'data_name': 'MGDA2',
           'data_source': "https://www.dropbox.com/sh/bj5ke9c1pqwpsct/AACqznMXzJj7epVBVDaSifnma?dl=0",
           'file_naming_scheme': ['Rec_date','Virus','GenoType','Sex','Age','Slice_Num','Cell_num','Cell_Type'],
           }

MGDA2_results = crispy_mouse_wrap(dataset,VC_prot,IC_prot,strat_cols=['Rec_date', 'Cell_Type'])

In [ ]:
dataset = {'data_name': 'NELL2',
           'data_source': "https://www.dropbox.com/sh/a3vrgx8n2lvs53w/AAAVi1gSXYaP3gY4PmvWlRbAa?dl=0",
           'file_naming_scheme': ['Rec_date','Virus','GenoType','Sex','Age','Slice_Num','Cell_num','Cell_Type'],
           }

NELL2_results = crispy_mouse_wrap(dataset,VC_prot,IC_prot,strat_cols=['Rec_date', 'Cell_Type'])

In [62]:
NELL2_results.keys()
strat_df_dict = NELL2_results['strat_df_dict']


# display( strat_df_dict[list(strat_df_dict.keys())[0]] )


labels = ['Rec_date', 'Virus', 'GenoType', 'Sex', 'Age', 'Slice_Num', 'Cell_num', 'Cell_Type']

possible_cols = list(strat_df_dict[list(strat_df_dict.keys())[0]].columns)
# print(possible_cols)

alt_strat_groups = ['Cm_pc_10.0',
                    'Ra_160.0',
                    'Rm_160.0',
                    'Cmq_160.0',
                    'Gain_(Hz/pA)',
                    'Rheobase',
                    'AP_thresh_US',
                    'fast_after_hyperpol',
                    'Spike_latency_(ms)',
                    'Input_Resistance_MO',
                    'Rmp_mV',
                    [c for c in possible_cols if 'Stim_Levels_(pA)' in c],
                    [c for c in possible_cols if 'Spike_Counts' in c],
                    [c for c in possible_cols if 'IV_Early_(V_stim)' in c],
                    [c for c in possible_cols if 'IV_Steady_State_(I_mean)' in c],
                    [c for c in possible_cols if 'IV_Early_(I_peak)' in c],
                    ]




alt_strat_dict = {}
for g in alt_strat_groups:
    new_df = pd.DataFrame()
    g_name = g
    for k,v in strat_df_dict.items():
        if isinstance(g, list):
            ''' iter the list'''
            g_name = g[0]
        else:
            new_col_name = g+'_'+k            
            clean_ser = v[g].reset_index().drop(labels='index',axis=1,inplace=False)

            len_diff =  len(clean_ser)-len(new_df)
            if len_diff>0:
                blank_df = pd.DataFrame( index=range(len_diff),columns=new_df.columns)
                new_df = new_df.append(blank_df,ignore_index=True)
            new_df[new_col_name]=clean_ser
    alt_strat_dict[g_name] = new_df
            


def new_single_col( new_df,cur_group)


print(alt_strat_dict.keys())
display( alt_strat_dict['Cm_pc_10.0'] )
alt_strat_dict['Cm_pc_10.0'].to_csv('test.csv')

dict_keys(['Cm_pc_10.0', 'Ra_160.0', 'Rm_160.0', 'Cmq_160.0', 'Gain_(Hz/pA)', 'Rheobase', 'AP_thresh_US', 'fast_after_hyperpol', 'Spike_latency_(ms)', 'Input_Resistance_MO', 'Rmp_mV', 'Stim_Levels_(pA)_0.0', 'Spike_Counts_0.0', 'IV_Early_(V_stim)_-130.0', 'IV_Steady_State_(I_mean)_-130.0_pApF', 'IV_Early_(I_peak)_-130.0_pApF'])


,Cm_pc_10.0__2022x08x04_2022x08x04_DGxPOS,Cm_pc_10.0__2022x08x04_2022x08x04_DGxNEG,Cm_pc_10.0__2022x08x09_2022x08x09_DGxPOS,Cm_pc_10.0__2022x08x09_2022x08x09_DGxNEG,Cm_pc_10.0__2022x08x09_2022x08x09_CA3xPOS,Cm_pc_10.0__2022x10x24_2022x10x24_CA3xPOS,Cm_pc_10.0__2022x10x24_2022x10x24_CA3xNEG,Cm_pc_10.0__2022x10x25_2022x10x25_CA3xPOS,Cm_pc_10.0__2022x10x25_2022x10x25_CA3xNEG,Cm_pc_10.0__2022x08x10_2022x08x10_DGxPOS,Cm_pc_10.0__2022x08x10_2022x08x10_DGxNEG
0,20.328385,20.006172,68.552933,16.578466,27.644784,162.184828,121.181946,89.853905,99.444768,19.09498,20.621392
1,35.170986,24.906178,39.647768,15.510334,NaN,86.70369,177.416788,66.370537,99.616286,15.46627,26.928566
2,25.134311,23.740864,NaN,NaN,NaN,228.479787,104.32332,415.390555,NaN,20.250761,75.218547
3,47.344681,51.339362,22.074578,11.281192,NaN,230.996467,391.99744,136.178275,NaN,NaN,21.944615
4,38.892001,24.800575,45.126846,18.558934,NaN,101.275053,87.731672,NaN,NaN,NaN,14.867899
5,NaN,NaN,21.102424,12.364011,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,24.045362,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,68.038781,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [49]:
len_diff = 3
[None for c in new_df.columns]*len_diff

[]

In [61]:
blank_df = pd.DataFrame( index=range(2),columns=range(3), )
display(blank_df)

,0,1,2
0,NaN,NaN,NaN
1,NaN,NaN,NaN
